<a href="https://colab.research.google.com/github/adriele07/tb3_arquitetura_de_computadores/blob/main/TP3_OC1_Simulador_de_Mem%C3%B3ria_Cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trabalho Prático 3 - Simulador de Memória Cache**

Neste trabalho, vocês deveram desenvolver um programa que funcione como um **simulador de memória cache**. O objetivo deste trabalho é proporcionar a prática dos conceitos de **mapeamento direto, associatividade por conjunto e associatividade completa** que foram abordados durante o curso. A prática desses conceitos será essencial para compreender as diferenças entre os métodos de mapeamento e como eles afetam a eficiência e o desempenho da memória cache.

Para simplificar o escopo do trabalho, o simulador não tratará os dados reais armazenados na memória. Em outras palavras, apenas o gerenciamento e o posicionamento das linhas na memória cache serão simulados, ignorando o conteúdo dos blocos.


**Aluno(s):**

### **📚 Fundamentos de Cache**

Um endereço de memória (por exemplo: 0xDEADBEEF) precisa ser dividido em partes para sabermos onde ele será mapeado na cache. Sendo assim, temos:

1. A memória principal é dividida em blocos (do tamanho da linha de cache);

2. A cache é dividida em linhas;

3. Cada linha pode armazenar 1 bloco da memória principal;

4. As linhas são agrupadas em conjuntos, dependendo do grau de associatividade;

<br>

### **🔢 Exemplo prático**

Considere:
```
O tamanho total da cache: 4096
O tamanho de cada linha: 1024
Total de linhas: 4096 / 1024 = 4
```


**1. MAPEAMENTO DIRETO**

* 4 linhas ⇒ 4 grupos
* Cada endereço mapeia em um único grupo fixo
* Número de bits de índice = log2(4) = 2 bits
* Offset = log2(1024) = 10 bits
* TAG = 32 - 2 - 10 = 20 bits

  ```
  ENDEREÇO = [TAG = 20 BITs | ÍNDICE = 2 BITs | OFFSET = 10 BITs ]
  ```

**2. ASSOCIATIVIDADE POR CONJUNTO**

* 4 linhas / 2 linhas por grupo ⇒ 2 grupos
* Número de bits de índice = log2(2) = 1 bit
* Offset = log2(1024) = 10 bits
* TAG = 32 - 1 - 10 = 21 bits

  ```
  ENDEREÇO = [TAG = 21 BITs | ÍNDICE = 1 BIT | OFFSET = 10 BITs ]

**3. ASSOCIATIVIDADE COMPLETA**

* 4 linhas / 4 linhas por grupo ⇒ 1 grupo
* Número de bits de índice = 0 (não há indice)
* Offset = log2(1024) = 10 bits
* TAG = 32 - 10 = 22 bits

  ```
  ENDEREÇO = [TAG = 22 BITs | OFFSET = 10 BITs ]
  ```


### **Especificação do simulador**

Neste trabalho, o objetivo é desenvolver um programa que funcione como um simulador de memória cache. O simulador será responsável por gerenciar uma cache de apenas um nível, processando acessos à memória RAM, preenchendo as linhas da cache conforme necessário e contabilizando a quantidade de HITS e MISSES ocorridos durante a execução. O programa deverá verificar se um determinado bloco de memória já está presente na cache (HIT) ou se precisa ser carregado da memória principal (MISS). Além disso, o simulador determinará em qual linha da cache um bloco de memória será armazenado, quando necessário.


Para simplificar o escopo do trabalho, o simulador não tratará os dados reais armazenados na memória. Em outras palavras, apenas o gerenciamento e o posicionamento das linhas na memória cache serão simulados, ignorando o conteúdo dos blocos.


O simulador deverá ser desenvolvido em Python, podendo utilizar qualquer biblioteca adicional que os alunos considerem necessária.


**Parâmetros de entrada:**



1. **O tamanho total da cache, em bytes**. Se tivermos uma cache de 4KB, por exemplo, iremos digitar 4096.
2. **O tamanho de cada linha, em bytes**. Uma linha de 1KB, por exemplo, seria entrada digitando-se 1024.
3. **O tamanho de cada grupo, em unidades**. Pensando em uma memória cache de 4KB, e páginas de 1KB, teremos 4 linhas. Se tivermos uma linha por grupo, teremos um sistema de mapeamento direto.  Se tivermos 4 por grupo, teremos um sistema de associatividade completa.




**Parâmetros de saida:**


A saída do simulador desenvolvido deverá ser salva em uma estrutura de lista de dicionários como o nome `history`, onde cada dicionário representa um acesso individual à cache, contendo as seguintes informações:


* **addr:** Endereço de memória acessado, no formato hexadecimal de 32 bits (exatamente 8 dígitos com prefixo 0x e letras maiúsculas). Esse valor corresponde ao identificador do bloco, obtido a partir do endereço original com os bits de deslocamento e, se aplicável, de conjuntos ignorados.


* **group:** Número do grupo (ou conjunto) da cache ao qual o bloco foi mapeado. Em caches com associatividade total (fully associative), este valor será sempre 0.


* **idx_line:** Índice da linha dentro do grupo utilizada para armazenar o bloco acessado.


* **result:** Resultado da operação de leitura na cache. Pode ser "HIT" se o bloco já estiver presente na cache, ou "MISS" se for necessário carregá-lo da memória principal.





### **Parâmetros fixos do simulador**


1. O espaço de endereçamento será de **32 bits**;
2. Os endereços sempre referenciam **bytes**, e não palavras;
3. A política de substituição de páginas é a **FIFO (First In First Out)**;
4. A alocação de linhas em um conjunto **seguirá a ordem**. Ou seja, se tivermos um conjunto de tamanho dois, vamos primeiro armazenar o bloco na primeira linha do conjunto, e em seguida na segunda;
5. As linhas de um conjunto serão armazenadas de **forma consecutiva** na cache (uma atrás da outra);
6. Teremos **menos de mil linhas** de cache.


# **Simulador de Memória Cache**

In [1]:
### Desenvolva seu código aqui
# Função principal do simulador de memória cache
import math
from collections import deque

def simulador_cache(cache_size, line_size, associativity, enderecos):
    history = []
    num_lines = cache_size // line_size
    num_groups = num_lines // associativity
    offset_bits = int(math.log2(line_size))
    group_bits = int(math.log2(num_groups)) if num_groups > 1 else 0

    # Inicializa a cache: lista de deques (um para cada grupo)
    cache = [deque(maxlen=associativity) for _ in range(num_groups)]

    for addr_str in enderecos:
        addr_int = int(addr_str, 16)
        # Remove bits de offset
        bloco = addr_int >> offset_bits
        # Calcula grupo (conjunto)
        if num_groups > 1:
            group = bloco & (num_groups - 1)
        else:
            group = 0
        # Remove bits do grupo para obter o identificador do bloco
        bloco_id = bloco >> group_bits
        # Formata identificador do bloco para 8 dígitos hexadecimais
        bloco_id_str = f"0x{bloco_id:08X}"
        # Verifica se está na cache (HIT)
        linhas = cache[group]
        hit = False
        idx_line = None
        for idx, tag in enumerate(linhas):
            if tag == bloco_id:
                hit = True
                idx_line = idx
                break
        if hit:
            result = "HIT"
        else:
            result = "MISS"
            # FIFO: adiciona o novo bloco
            if len(linhas) < associativity:
                linhas.append(bloco_id)
                idx_line = len(linhas) - 1
            else:
                linhas.popleft()
                linhas.append(bloco_id)
                idx_line = len(linhas) - 1
        history.append({
            'addr': bloco_id_str,
            'group': group,
            'idx_line': idx_line,
            'result': result
        })
    return history

# Exemplo de uso:
# history = simulador_cache(cache_size, line_size, associativity, entrada1)
# imprimir_resultados(history, cache_size, line_size, associativity)

### **Função de visualizacão da Cache**

In [2]:
import pandas as pd
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

def imprimir_resultados(historico, cache_size, line_size, associativity):
    df = pd.DataFrame(historico)

    num_lines = cache_size // line_size
    num_groups = num_lines // associativity

    # === Histórico ===
    print("\n=== Histórico de Acessos ===")
    def color_result(val):
        if val == "HIT":
            return "background-color: lightgreen; text-align: center"
        elif val == "MISS":
            return "background-color: lightcoral; text-align: center"
        else:
            return ""

    styled_df = df.style.map(color_result, subset=["result"]).set_properties(**{
        'text-align': 'center'
    }).set_table_attributes('style="margin: auto"')

    display(styled_df)

    # === Contadores ===
    hits = (df["result"] == "HIT").sum()
    misses = (df["result"] == "MISS").sum()

    print(f"\nHITs: {hits}")
    print(f"MISSes: {misses}")

    # === Estado final da cache ===
    ultima_ocorrencia = (
        df[df["result"] == "MISS"]
        .drop_duplicates(subset=["group", "idx_line"], keep='last')
    )

    estado_cache = []
    for group in range(num_groups):
        for idx_line in range(associativity):
            match = ultima_ocorrencia.query("group == @group and idx_line == @idx_line")
            addr = match.iloc[0]["addr"] if not match.empty else "-"
            estado_cache.append({
                'group': group,
                'idx_line': idx_line,
                'addr': addr
            })

    estado_df = pd.DataFrame(estado_cache)

    print("\n=== Estado Final da Cache ===")
    styled_estado = estado_df.style.map(
        lambda v: "background-color: lightgray; text-align: center" if v == "-" else "text-align: center",
        subset=["addr"]
    ).set_properties(**{
        'text-align': 'center'
    }).set_table_attributes('style="margin: auto"')

    display(styled_estado)

    # === Interatividade: passo a passo ===
    print("\n=== Evolução da Cache (passo a passo) ===")

    slider = widgets.IntSlider(
        value=0,
        min=0,
        max=len(df),
        step=1,
        description='Passo:',
        continuous_update=False
    )

    output = widgets.Output()

    def atualizar_cache(passo):
        with output:
            clear_output(wait=True)

            if passo == 0:
                print("⚠️ Nenhum acesso realizado ainda.")
                return

            parcial = df.iloc[:passo]
            ult_parcial = (
                parcial[parcial["result"] == "MISS"]
                .drop_duplicates(subset=["group", "idx_line"], keep='last')
            )

            estado_parcial = []
            for group in range(num_groups):
                for idx_line in range(associativity):
                    match = ult_parcial.query("group == @group and idx_line == @idx_line")
                    addr = match.iloc[0]["addr"] if not match.empty else "-"
                    estado_parcial.append({
                        'group': group,
                        'idx_line': idx_line,
                        'addr': addr
                    })

            parcial_df = pd.DataFrame(estado_parcial)

            print(f"📍 Acesso {passo}/{len(df)}: addr = {df.iloc[passo-1]['addr']} | result = {df.iloc[passo-1]['result']}")
            styled_parcial = parcial_df.style.map(
                lambda v: "background-color: lightgray; text-align: center" if v == "-" else "text-align: center",
                subset=["addr"]
            ).set_properties(**{
                'text-align': 'center'
            }).set_table_attributes('style="margin: auto"')

            display(styled_parcial)

    slider.observe(lambda change: atualizar_cache(change["new"]), names="value")

    display(slider, output)
    atualizar_cache(0)

### **EXEMPLO 1**

In [3]:
# Parâmetros de entrada
cache_size = 4096
line_size = 1024
associativity = 4

entrada1 = [
    '0x0CB886CA',
    '0x06BC89BA',
    '0x379CBAD1',
    '0x28F3123B',
    '0x06F98ED5'
]

history = simulador_cache(cache_size, line_size, associativity, entrada1)

imprimir_resultados(history, cache_size, line_size, associativity)


=== Histórico de Acessos ===


,addr,group,idx_line,result
0,0x00032E21,0,0,MISS
1,0x0001AF22,0,1,MISS
2,0x000DE72E,0,2,MISS
3,0x000A3CC4,0,3,MISS
4,0x0001BE63,0,3,MISS



HITs: 0
MISSes: 5

=== Estado Final da Cache ===


,group,idx_line,addr
0,0,0,0x00032E21
1,0,1,0x0001AF22
2,0,2,0x000DE72E
3,0,3,0x0001BE63



=== Evolução da Cache (passo a passo) ===


IntSlider(value=0, continuous_update=False, description='Passo:', max=5)

Output()

### **EXEMPLO 2**

In [4]:
# Parâmetros de entrada
cache_size = 4096
line_size = 1024
associativity = 4

entrada2 = [
  '0x0CB886CA',
  '0x06BC89BA',
  '0x379CBAD1',
  '0x28F3123B',
  '0x06F98ED5',
  '0x06F98ED8',
  '0x06BC89BA'
]

history = simulador_cache(cache_size, line_size, associativity, entrada2)

imprimir_resultados(history, cache_size, line_size, associativity)


=== Histórico de Acessos ===


,addr,group,idx_line,result
0,0x00032E21,0,0,MISS
1,0x0001AF22,0,1,MISS
2,0x000DE72E,0,2,MISS
3,0x000A3CC4,0,3,MISS
4,0x0001BE63,0,3,MISS
5,0x0001BE63,0,3,HIT
6,0x0001AF22,0,0,HIT



HITs: 2
MISSes: 5



=== Estado Final da Cache ===


,group,idx_line,addr
0,0,0,0x00032E21
1,0,1,0x0001AF22
2,0,2,0x000DE72E
3,0,3,0x0001BE63



=== Evolução da Cache (passo a passo) ===


IntSlider(value=0, continuous_update=False, description='Passo:', max=7)

Output()

### **EXEMPLO 3**

In [5]:
# Parâmetros de entrada
cache_size = 4096
line_size = 1024
associativity = 4

entrada3 = [
  '0x0CB886CA',
  '0x06BC89BA',
  '0x379CBAD1',
  '0x28F3123B',
  '0x06F98ED5',
  '0x06F98ED8',
  '0x17AA08A2'
]

history = simulador_cache(cache_size, line_size, associativity, entrada3)

imprimir_resultados(history, cache_size, line_size, associativity)


=== Histórico de Acessos ===


,addr,group,idx_line,result
0,0x00032E21,0,0,MISS
1,0x0001AF22,0,1,MISS
2,0x000DE72E,0,2,MISS
3,0x000A3CC4,0,3,MISS
4,0x0001BE63,0,3,MISS
5,0x0001BE63,0,3,HIT
6,0x0005EA82,0,3,MISS



HITs: 1
MISSes: 6

=== Estado Final da Cache ===


,group,idx_line,addr
0,0,0,0x00032E21
1,0,1,0x0001AF22
2,0,2,0x000DE72E
3,0,3,0x0005EA82



=== Evolução da Cache (passo a passo) ===


IntSlider(value=0, continuous_update=False, description='Passo:', max=7)

Output()

### **EXEMPLO 4**

In [6]:
# Parâmetros de entrada
cache_size = 4096
line_size = 1024
associativity = 4

entrada4 = [
  '0x0CB886CA',
  '0x06BC89BA',
  '0x379CBAD1',
  '0x28F3123B',
  '0x06F98ED5',
  '0x06F98ED8',
  '0x17AA08A2',
  '0x06BC89BA'
]

history = simulador_cache(cache_size, line_size, associativity, entrada4)

imprimir_resultados(history, cache_size, line_size, associativity)


=== Histórico de Acessos ===


,addr,group,idx_line,result
0,0x00032E21,0,0,MISS
1,0x0001AF22,0,1,MISS
2,0x000DE72E,0,2,MISS
3,0x000A3CC4,0,3,MISS
4,0x0001BE63,0,3,MISS
5,0x0001BE63,0,3,HIT
6,0x0005EA82,0,3,MISS
7,0x0001AF22,0,3,MISS



HITs: 1
MISSes: 7

=== Estado Final da Cache ===


,group,idx_line,addr
0,0,0,0x00032E21
1,0,1,0x0001AF22
2,0,2,0x000DE72E
3,0,3,0x0001AF22



=== Evolução da Cache (passo a passo) ===


IntSlider(value=0, continuous_update=False, description='Passo:', max=8)

Output()

### **EXEMPLO 5**

In [7]:
# Parâmetros de entrada
cache_size = 4096
line_size = 1024
associativity = 2

entrada5 = [
  '0x0CB886CA',
  '0x06BC89BA'
]

history = simulador_cache(cache_size, line_size, associativity, entrada5)

imprimir_resultados(history, cache_size, line_size, associativity)


=== Histórico de Acessos ===


,addr,group,idx_line,result
0,0x00019710,1,0,MISS
1,0x0000D791,0,0,MISS



HITs: 0
MISSes: 2

=== Estado Final da Cache ===


,group,idx_line,addr
0,0,0,0x0000D791
1,0,1,-
2,1,0,0x00019710
3,1,1,-



=== Evolução da Cache (passo a passo) ===


IntSlider(value=0, continuous_update=False, description='Passo:', max=2)

Output()

### **EXEMPLO 6**

In [8]:
# Parâmetros de entrada
cache_size = 4096
line_size = 1024
associativity = 1

entrada6 = [
  '0x0CB886CA',
  '0x06BC89BA',
  '0x379CBAD1',
  '0x28F3123B',
  '0x06F98ED5',
  '0x06F98ED8',
  '0x17AA08A2',
]

history = simulador_cache(cache_size, line_size, associativity, entrada6)

imprimir_resultados(history, cache_size, line_size, associativity)


=== Histórico de Acessos ===


,addr,group,idx_line,result
0,0x0000CB88,1,0,MISS
1,0x00006BC8,2,0,MISS
2,0x000379CB,2,0,MISS
3,0x00028F31,0,0,MISS
4,0x00006F98,3,0,MISS
5,0x00006F98,3,0,HIT
6,0x00017AA0,2,0,MISS



HITs: 1
MISSes: 6

=== Estado Final da Cache ===


,group,idx_line,addr
0,0,0,0x00028F31
1,1,0,0x0000CB88
2,2,0,0x00017AA0
3,3,0,0x00006F98



=== Evolução da Cache (passo a passo) ===


IntSlider(value=0, continuous_update=False, description='Passo:', max=7)

Output()